# ETL 
Tomando en consideración la información provista por el gobierno de Canada, con respecto al consumo de combustible en los autos de gasolina y electricos, se procede a realizar la extracción y transformación de los datos para un posterior análisis.

Fuente: https://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64

Los archivos utilizados son:

1. MY2023 Fuel Consumption Ratings.csv: Contiene la información de las principales marcas de autos que consumen gasolina, especificamente en los modelos del año 2023, mostrando ciertas características inherentes al automóvil.
2. MY2012-2024 Battery Electric Vehicles.csv: Contiene la información de las principales marcas de autos eléctricos, desde los años 2012 al 2024, mostrando ciertas características inherentes al automóvil.

Adicional, se trabajará con el archivo Electric and Alternative Fuel Charging Stations.csv provisto por la empresa, en el cuál se muestra el listado de las estaciones de carga de combustible, ya sea de gasolina o electrico en los estados unidos.

In [ ]:
# Se cargan las librerías necesarias para el análisis
import pandas as pd # Librería para trabajar con dataframes
import numpy as np # Librería para usos estadísticos

# Archivo de autos de gasolina

In [32]:
# Se procede a leer el archivo de autos de gasolina
df_Autos_Gasolina = pd.read_csv('..\DataSets\MY2023 Fuel Consumption Ratings.csv', encoding="latin1")

In [33]:
# Ya que tenemos cargado nuestro archivo en el dataframe,
# nos enfocamos en eliminar las columnas que no utilizaremos en el análisis
# Se eliminan las columnas de Engine size, cylinders, transmission, fuel type, combined l/100 y combinen mpg
EliminarColumnas = ["Engine size (L)", "Cylinders",	"Transmission", "Fuel type", "Combined (L/100 km)", "Combined (mpg)"]
df_Autos_Gasolina.drop(EliminarColumnas, axis=1, inplace = True)

In [34]:
# Se procede a eliminar los registros duplicados
df_Autos_Gasolina.drop_duplicates(inplace=True)

In [35]:
# Agregamos las columnas que existen en el archivo de autos electricos para después unificar
# Además, se agrega una columna adicional para hacer mención que es un auto de gasolina
df_Autos_Gasolina['City (kWh/100 mi)'] = np.nan
df_Autos_Gasolina['Highway (kWh/100 mi)'] = np.nan
df_Autos_Gasolina['Range (mi)'] = np.nan
df_Autos_Gasolina['Recharge time (h)'] = np.nan
df_Autos_Gasolina['Type_Car'] = "Gasoline"

In [36]:
# Debido a que el sistema de métrico en estados unidos es diferente, se requiere hacer una transformación
# Por ende, vamos a pasar una función para convertir los litros por kilometro a galones por milla
def ConversionLKMaGMi(LitrosPorKilometro): # Recibimos como argumento el valor a transformar
    # Tomando en cuenta que que un litro es aproximadamente 0.264172 galones
    # Tenemos que multiplicar los litros recibidos por esta equivalencia, posteriormente dividir entre 100, ya que es la cantidad de kilometros en lo que está medida
    # De esta forma sacamos el valor unitario de cuantos galones por kilometro se gasta
    GalonesPorKilometro = LitrosPorKilometro * 0.264172 / 100 # Se aplica la formula
    MillasPorKilometro = 0.621371 # Esta es la equivalencia de cuantos kilometros hay en una mulla
    GalonesPorMilla = (GalonesPorKilometro * MillasPorKilometro) # Se realiza la multiplicación para 
    return GalonesPorMilla # Se retorna el valor calculado

# Creamos las nuevas columnas con los valores convertidos
df_Autos_Gasolina['City (Gal/mi)'] = df_Autos_Gasolina['City (L/100 km)'].apply(ConversionLKMaGMi)
df_Autos_Gasolina['Highway (Gal/mi)'] = df_Autos_Gasolina['Highway (L/100 km)'].apply(ConversionLKMaGMi)

In [37]:
# Se eliminan las columnas del sistema métrico anterior
EliminarColumnas = ["City (L/100 km)", "Highway (L/100 km)"]
df_Autos_Gasolina.drop(EliminarColumnas, axis=1, inplace = True)
df_Autos_Gasolina.info()

<class 'pandas.core.frame.DataFrame'>
Index: 831 entries, 0 to 832
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Model year            831 non-null    int64  
 1   Make                  831 non-null    object 
 2   Model                 831 non-null    object 
 3   Vehicle class         831 non-null    object 
 4   CO2 emissions (g/km)  831 non-null    int64  
 5   CO2 rating            831 non-null    int64  
 6   Smog rating           831 non-null    int64  
 7   City (kWh/100 mi)     0 non-null      float64
 8   Highway (kWh/100 mi)  0 non-null      float64
 9   Range (mi)            0 non-null      float64
 10  Recharge time (h)     0 non-null      float64
 11  Type_Car              831 non-null    object 
 12  City (Gal/mi)         831 non-null    float64
 13  Highway (Gal/mi)      831 non-null    float64
dtypes: float64(6), int64(4), object(4)
memory usage: 97.4+ KB


# Archivo de autos eléctricos

In [38]:
# Se procede a leer el archivo de autos electricos
df_Autos_Electricos = pd.read_csv('..\DataSets\MY2012-2024 Battery Electric Vehicles.csv', encoding="utf-8")

In [39]:
# Ya que tenemos cargado nuestro archivo en el dataframe,
# nos enfocamos en eliminar las columnas que no utilizaremos en el análisis
# Se eliminan las columnas de Engine size, cylinders, transmission, fuel type, combined l/100 y combinen mpg
EliminarColumnas = ["Motor (kW)", "Transmission", "Fuel type", "Combined (kWh/100 km)", "Combined (Le/100 km)"]
df_Autos_Electricos.drop(EliminarColumnas, axis=1, inplace = True)

In [40]:
# Se procede a eliminar los registros duplicados
df_Autos_Electricos.drop_duplicates(inplace=True)

In [41]:
# Renombramos las columnas para unificar títulos
df_Autos_Electricos = df_Autos_Electricos.rename(columns={'City (Le/100 km)': 'City (L/100 km)', 'Highway (Le/100 km)': 'Highway (L/100 km)', 'CO2 rating ': 'CO2 rating'})

In [42]:
# Agregamos una columna adicional para mencionar que es un auto electrico
df_Autos_Electricos['Type_Car'] = "Electric"

In [43]:
# Filtramos nuestro archivo para mostrar únicamente los valores que pertenezcan a modelos del año 2023
df_Autos_Electricos = df_Autos_Electricos[df_Autos_Electricos['Model year'] == 2023]

In [44]:
# Igual que en el archivo de autos de combustible, se procede a convertir los Litros/Kilómetro a Galones/Milla
df_Autos_Electricos['City (Gal/mi)'] = df_Autos_Electricos['City (L/100 km)'].apply(ConversionLKMaGMi)
df_Autos_Electricos['Highway (Gal/mi)'] = df_Autos_Electricos['Highway (L/100 km)'].apply(ConversionLKMaGMi)
# Transformamos los valores de Kilometros a millas haciendo la multiplicación por su equivalencia
df_Autos_Electricos['Range (mi)'] = df_Autos_Electricos['Range (km)'] * 0.621371
df_Autos_Electricos["City (kWh/100 mi)"] = df_Autos_Electricos["City (kWh/100 km)"] * 1.60934
df_Autos_Electricos["Highway (kWh/100 mi)"] = df_Autos_Electricos["Highway (kWh/100 km)"] * 1.60934

# Se eliminan las columnas del sistema métrico anterior
EliminarColumnas = ["City (L/100 km)", "Highway (L/100 km)", "Range (km)", "City (kWh/100 km)", "Highway (kWh/100 km)"]
df_Autos_Electricos.drop(EliminarColumnas, axis=1, inplace = True)

In [45]:
# Verificamos que los campos no tengan valores nulos
df_Autos_Electricos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159 entries, 311 to 469
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Model year            159 non-null    int64  
 1   Make                  159 non-null    object 
 2   Model                 159 non-null    object 
 3   Vehicle class         159 non-null    object 
 4   CO2 emissions (g/km)  159 non-null    int64  
 5   CO2 rating            159 non-null    float64
 6   Smog rating           159 non-null    float64
 7   Recharge time (h)     159 non-null    float64
 8   Type_Car              159 non-null    object 
 9   City (Gal/mi)         159 non-null    float64
 10  Highway (Gal/mi)      159 non-null    float64
 11  Range (mi)            159 non-null    float64
 12  City (kWh/100 mi)     159 non-null    float64
 13  Highway (kWh/100 mi)  159 non-null    float64
dtypes: float64(8), int64(2), object(4)
memory usage: 18.6+ KB


In [46]:
# Se procede a unir ambos dataframe en uno solo para posteriormente exportarlo a un archivo csv limpio y unificado
df_Autos = pd.merge(df_Autos_Gasolina, df_Autos_Electricos, on=['Model year', 'Make', 'Model', 'Vehicle class', 'City (Gal/mi)', 'Highway (Gal/mi)', 'CO2 emissions (g/km)', 'CO2 rating', 'Smog rating', 'City (kWh/100 mi)', 'Highway (kWh/100 mi)', 'Range (mi)', 'Recharge time (h)', 'Type_Car'], how="outer")

In [47]:
# Verificamos los valores de nuestro dataframe unido
df_Autos.head()

,Model year,Make,Model,Vehicle class,CO2 emissions (g/km),CO2 rating,Smog rating,City (kWh/100 mi),Highway (kWh/100 mi),Range (mi),Recharge time (h),Type_Car,City (Gal/mi),Highway (Gal/mi)
0,2023,Acura,Integra,Full-size,167,6.0,7.0,NaN,NaN,NaN,NaN,Gasoline,0.012968,0.010341
1,2023,Acura,Integra A-SPEC,Full-size,172,6.0,7.0,NaN,NaN,NaN,NaN,Gasoline,0.013296,0.010670
2,2023,Acura,Integra A-SPEC,Full-size,181,6.0,6.0,NaN,NaN,NaN,NaN,Gasoline,0.014609,0.010670
3,2023,Acura,MDX SH-AWD,Sport utility vehicle: Small,263,4.0,5.0,NaN,NaN,NaN,NaN,Gasoline,0.020683,0.015430
4,2023,Acura,MDX SH-AWD Type S,Sport utility vehicle: Standard,291,4.0,5.0,NaN,NaN,NaN,NaN,Gasoline,0.022653,0.018385


In [48]:
# Exportamos el resultado de nuestro dataframe en un archivo de tipo csv
df_Autos.to_csv('..\DataSets\Autos2023.csv', index=False)

Diccionario de datos Autos 2023:
Este archivo, contiene la 
Especificamos el significado de cada una de las columnas.

Model year: El año al que pertenece el modelo del auto
Make: EL fabricante del auto
Model: El modelo del auto
Vehicle class: El tipo de vehículo
City (Gal/mi): Cantidad de galones que consume por milla recorrida dentro de la ciudad
Highway (Gal/mi): Cantidad de galones que consume por milla recorrida en carreteras
CO2 emissions (g/km): Emisión de CO2 emitido a la atmosfera en una medida de gramos por kilometro recorrido
CO2 rating: Las emisiones de dióxido de carbono del tubo de escape del vehículo se clasifican en una escala del 1 (peor) al 10 (mejor).
Smog rating: Las emisiones del tubo de escape del vehículo de contaminantes que forman smog se clasifican en una escala del 1 (peor) al 10 (mejor).
City (kWh/100 mi): Cantidad de kiloWhats Hora que se consumen por cada 100 millas recorridos dentro de la ciudad
Highway (kWh/100 mi): Cantidad de kiloWhats Hora que se consumen por cada 100 millas recorridos dentro de carreteras
Range (mi): Millas que puede recorrer el auto con una carga completa de su batería
Recharge time (h): Cantidade de horas que se requieren para cargar por completo la batería del auto
Type_Car: Define si el auto es de gasolina o es eléctrico

# Archivo de estaciones de carga

In [49]:
# Se procede a leer el archivo de autos electricos
df_Estaciones_Carga = pd.read_csv('..\DataSets\Electric and Alternative Fuel Charging Stations.csv', encoding="utf-8")

C:\Users\Windows10\AppData\Local\Temp\ipykernel_8812\3251425989.py:2: DtypeWarning: Columns (6,16,20,31,33,36,39,40,41,43,46,52,53,55,57,58,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df_Estaciones_Carga = pd.read_csv('..\DataSets\Electric and Alternative Fuel Charging Stations.csv', encoding="utf-8")


In [50]:
# Este archivo contiene todas las estaciones de carga de combustible de gasolina y electricos en USA, 
# Por lo que primero vamos a filtrar únicamente las estaciones de carga para autos electricos en el estado de Nueva York
df_Estaciones_Carga = df_Estaciones_Carga[df_Estaciones_Carga["Fuel Type Code"] == "ELEC"]
df_Estaciones_Carga = df_Estaciones_Carga[df_Estaciones_Carga["State"] == "NY"]

In [51]:
# Este archivo contiene muchas columnas que no vamos a ocupar, 
# Por lo que se procede a eliminar las columnas que no serán parte de nuestro análisis

# Primero, definimos el nombre de las columnas que sí vamos a utilizar
ConservarColumnas = ["Station Name", "Street Address", "City", "State", "ZIP", "Station Phone", "Latitude", "Longitude", "ID", "EV Pricing"]
df_Estaciones_Carga = df_Estaciones_Carga[ConservarColumnas]

In [52]:
# Se procede a eliminar los registros duplicados
df_Estaciones_Carga.drop_duplicates(inplace=True)

In [53]:
# Renombramos la columna de precio para posteriormente agrupar sus valores en una nueva columna
df_Estaciones_Carga = df_Estaciones_Carga.rename(columns={'EV Pricing': 'EV Pricing Detail'})

In [54]:
# Vamos a crear una nueva nueva columna, en la que vamos a agrupar el precio de la carga de combustible en 3 valores
# Carga gratis, con costo o sin datos
# Para ello, vamos a crear una función, la cuál va a recibir como parámetro un valor, 
# mismo que se va a analizar utilizando condiciones y definir cuál será el valor que se asignará a la nueva columna

def Asignar_Precio(Valor):
    if pd.isna(Valor) or Valor == "":
        return np.nan
    elif "Free" in Valor:
        return "Free"
    elif "Free; donations encouraged" in Valor:
        return "Free"
    else:
        return "Cost"

In [55]:
# Utilizando la función anterior, se procede a crear la nueva columna de precio agrupado
df_Estaciones_Carga['EV Price'] = df_Estaciones_Carga['EV Pricing Detail'].apply(Asignar_Precio)

In [56]:
# Exportamos el resultado de nuestro dataframe en un archivo de tipo csv
df_Estaciones_Carga.to_csv('..\DataSets\Estaciones_Carga_NY.csv', index=False)

# Diccionario de datos
Se crea un diccionario de datos para definir el significado de cada una de las columnas del archivo

Station Name: Nombre de la estación de carga
Street Address: Calle en la que se ubica la estación
City: Ciudad a la que pertenece la estación de carga
State: Estado en donde se encuentra la estación de carga
ZIP: Codigo postal de la estación de carga
Station Phone: Teléfono de la estación de carga
Latitude: Latitud geográfica de la estación de carga
Longitude: Longitud geográfica de la estación de carga
ID: Identificador de la estación de carga
EV Pricing Detail: Detalle del precio que cobran en la estación de carga
EV Price: Clasificación de cobro de la estación, para saber si la carga tiene un costo es gratuita
